# Leave-One-Subject-Out Evaluation

In [1]:
import numpy as np
import cv2
import dlib
from imutils import face_utils
import glob
import pickle
from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn import metrics
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

### 3D Model points for SolvePnP
We will approximate the 3D points of the following face parts with the correspoinding coordinates. This is a general model of the human face and we do not need to worry much about absolute accuracy.

In [2]:
# 3D model points.
model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                            (0.0, -330.0, -65.0),        # Chin
                            (-225.0, 170.0, -135.0),     # Left eye left corner
                            (225.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner
                         
                        ])

### Load Dataset

In [30]:
# Number of training examples to use(0-2806)
DATASET_SIZE = 2758
DEBUG = False

# Load the dataset
with open('data_cleaned.json') as json_file:
    data_all = json.load(json_file)

# Extract the keys in sorted order
keys = sorted(data_all)

# Convert python list to np array
keys = np.asarray(keys)

random_key = np.random.randint(DATASET_SIZE, size=1)
# Extract the UUID of the randomly chosen key and leave out every key with that UUID
uuid_excluded = keys[random_key][0].split('/')[0]

indices_excluded = []
keys_excluded = []
for i in range(DATASET_SIZE):
    key = keys[i]
    uuid = key.split('/')[0]
    if(uuid == uuid_excluded):
        indices_excluded.append(i)
        keys_excluded.append(key)
# keys = np.delete(keys, indices_excluded)
print(keys.shape)
DATASET_SIZE = keys.shape[0]

(2758,)


In [31]:
X = np.zeros((DATASET_SIZE, 6, 1))
y = np.zeros(DATASET_SIZE)

for i in range(DATASET_SIZE):
    key = keys[i]

    # Approximate camera intrinsic parameters
    im = cv2.imread('dataset/' + key)   # This imread is time consuming! Another way?
    size = im.shape
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                             [[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )
    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    
    landmarks = data_all[key]['landmarks']
    
    # Grab the 2D coordinates of our six sample points
    image_points = np.array([
        (landmarks[33]['x'], landmarks[33]['y']) ,     # Nose tip
        (landmarks[8]['x'], landmarks[8]['y']),     # Chin
        (landmarks[36]['x'], landmarks[36]['y']),     # Left eye left corner
        (landmarks[45]['x'], landmarks[45]['y']),     # Right eye right corner
        (landmarks[48]['x'], landmarks[48]['y']),     # Left Mouth corner
        (landmarks[54]['x'], landmarks[54]['y'])      # Right mouth corner
    ], dtype="double")
    
    # Solve the PnP problem with the parameters specified above
    # and obtain rotation and translation vectors
    (success, rotation_vector, translation_vector) = cv2.solvePnP(
        model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
        )
    
    X[i, :] = np.concatenate((rotation_vector, translation_vector), axis=0)
    # Check if it is positive or negative example
    output = key.split('/')[1]
    if(output == 'positive'):
        y[i] = 1
    elif(output == 'negative'):
        y[i] = 0
    
    # DEBUGGING
    if(DEBUG and i <= 10):
        # Project a 3D point (0, 0, 1000.0) onto the image plane.
        # We use this to draw a line sticking out of the nose
        (nose_end_point2D, jacobian) = cv2.projectPoints(
            np.array([(0.0, 0.0, 500.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs
            )
        for p in image_points:
            cv2.circle(im, (int(p[0]), int(p[1])), 3, (0,0,255), -1)
        p1 = ( int(image_points[0][0]), int(image_points[0][1]) )
        p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]) )
        
        # Draw a line connecting the two points. This line must show
        # the direction out of the nose
        cv2.line(im, p1, p2, (255,0,0), 2)
        # Display image
        cv2.imshow("Output", im)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        print("Rotation Vector:\n {0}".format(rotation_vector))
        print("Translation Vector:\n {0}".format(translation_vector))

X = X.squeeze()
print(X.shape)
print(y.shape)

(2758, 6)
(2758,)


In [45]:
m = X[0:500].mean(axis=0)
std = X[0:500].std(axis=0)

X_scaled = (X - m)/std

### Train and Predict

In [87]:
 X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

In [88]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=1)
rf_classifier.fit(X_train, y_train)
y_pred_rf = rf_classifier.predict(X_test)

print('Test set accuracy for Random Forest: ', metrics.accuracy_score(y_test, y_pred_rf))

Test set accuracy for Random Forest:  0.7391304347826086


In [57]:
svm_classifier = svm.SVC(C=100, kernel='rbf', gamma='auto')
svm_classifier.fit(X_train, y_train)
y_pred_svm = svm_classifier.predict(X_test)

print('Test set accuracy for SVM: ', metrics.accuracy_score(y_test, y_pred_svm))

Test set accuracy for SVM:  0.6394927536231884


In [39]:
print(len(keys_excluded))

36


In [76]:
X_eval = np.zeros((len(keys_excluded), 6, 1))
y_eval = np.zeros(len(keys_excluded))

for i in range(len(keys_excluded)):
    key = keys_excluded[i]
        
    # Approximate camera intrinsic parameters
    im = cv2.imread('dataset/' + key)   # This imread is time consuming! Another way?
    size = im.shape
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                             [[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )
    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion

    landmarks = data_all[key]['landmarks']
    
    # Grab the 2D coordinates of our six sample points
    image_points = np.array([
        (landmarks[33]['x'], landmarks[33]['y']) ,     # Nose tip
        (landmarks[8]['x'], landmarks[8]['y']),     # Chin
        (landmarks[36]['x'], landmarks[36]['y']),     # Left eye left corner
        (landmarks[45]['x'], landmarks[45]['y']),     # Right eye right corner
        (landmarks[48]['x'], landmarks[48]['y']),     # Left Mouth corner
        (landmarks[54]['x'], landmarks[54]['y'])      # Right mouth corner
    ], dtype="double")
    
    # Solve the PnP problem with the parameters specified above
    # and obtain rotation and translation vectors
    (success, rotation_vector, translation_vector) = cv2.solvePnP(
        model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
        )
    
    # Check if it is positive or negative example
    output = key.split('/')[1]
    if(output == 'positive'):
        y_eval[i] = 1
    elif(output == 'negative'):
        y_eval[i] = 0
    
    if(False and i<20):
        # Project a 3D point (0, 0, 1000.0) onto the image plane.
        # We use this to draw a line sticking out of the nose
        (nose_end_point2D, jacobian) = cv2.projectPoints(
            np.array([(0.0, 0.0, 500.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs
            )
        for p in image_points:
            cv2.circle(im, (int(p[0]), int(p[1])), 3, (0,0,255), -1)
        p1 = ( int(image_points[0][0]), int(image_points[0][1]) )
        p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]) )
        
        # Draw a line connecting the two points. This line must show
        # the direction out of the nose
        cv2.line(im, p1, p2, (255,0,0), 2)
        # Display image
        cv2.imshow(output, im)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        print("Rotation Vector:\n {0}".format(rotation_vector))
        print("Translation Vector:\n {0}".format(translation_vector))
    
    X_eval[i, :] = np.concatenate((rotation_vector, translation_vector), axis=0)


X_eval = X_eval.squeeze()
print(X_eval.shape, y_eval.shape)

(36, 6) (36,)


In [77]:
m_eval = X_eval.mean(axis=0)
std_eval = X_eval.std(axis=0)

X_eval = (X_eval - m_eval)/std_eval

In [78]:
# rand_indices = np.random.randint(len(keys_excluded), size=len(keys_excluded))
# X_eval_scaled = X_eval_scaled[rand_indices, :]
# y_eval = y_eval[rand_indices]
# print(X_eval_scaled.shape)

In [89]:
y_eval_rf = rf_classifier.predict(X_eval)

print('Test set accuracy for Random Forest: ', metrics.accuracy_score(y_eval, y_eval_rf))

Test set accuracy for Random Forest:  0.6388888888888888
